In [4]:
import sys
import os 
import findspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, BooleanType, LongType, IntegerType
os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1 pyspark-shell"


In [9]:
findspark.init()
findspark.find()
spark = SparkSession.builder.appName("practice").master("local").getOrCreate()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1108)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1094)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:579)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1647)
	at org.apache.spark.SparkContext.$anonfun$new$13(SparkContext.scala:514)
	at org.apache.spark.SparkContext.$anonfun$new$13$adapted(SparkContext.scala:514)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:514)
	at org.apache.spark.api.java.JavaSparkC

In [7]:
df = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:29092, localhost:39092").option("subscribe", "topic").load()

NameError: name 'spark' is not defined

In [15]:
df1 = df.withColumn("value", df["value"].cast(StringType()))

In [16]:
display(df1)

DataFrame[key: binary, value: string, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [18]:
output = df1.writeStream.outputMode("append").format('parquet').option("checkpointLocation", "./kafka_stream.parquet").option("path", './kafka_stream.parquet').start()

22/12/15 15:24:12 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
22/12/15 15:24:12 WARN StreamingQueryManager: Stopping existing streaming query [id=6f20ba3f-2541-4d5b-9e92-ebd48785304a, runId=c4b1e9d5-ca4a-4ddc-9cd0-b1831309b99b], as a new run is being started.


In [ ]:
output.awaitTermination()